### Dependencies

In [9]:
import cv2
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.python import pywrap_tensorflow

sys.path.append('..')

import video_input as vi
import model as model
import utils as utils

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
params = utils.yaml_to_dict('../config.yml')
params['data_dir'] = os.path.join('..', params['data_dir'])
params['videos_folder'] = os.path.join('..', params['videos_folder'])

In [11]:
params

{'data_dir': '../data',
 'resize': [224, 224],
 'shuffle': True,
 'skip_frames': 10,
 'temp_dir': '.temp',
 'videos_folder': '../data/videos'}

### Set test image

In [12]:
img_path = os.path.join('..',params['temp_dir'], 'test_img.jpg')
img_width = 224
img_height = 224

test_img = cv2.imread(img_path)
print('Original Size',test_img.shape)

img_resize = cv2.resize(test_img,(img_width,img_height))
img_resize = img_resize[None, ...]

img_normalized = (img_resize / 255.0)
print('Resized and normalized image', img_normalized.shape)

Original Size (2448, 3264, 3)
Resized and normalized image (1, 224, 224, 3)


### Load pretrained model

In [13]:
pretrain_model = model._initialize_pretrained_model()

### Make predictions

In [14]:
video_path = os.path.join(params['videos_folder'],'videoplayback.mp4')
video_gen = vi.load_video(video_path)

In [15]:
block4_pool_features = pretrain_model.predict(next(video_gen))

In [16]:
block4_pool_features.shape

(644, 5, 5, 1536)

In [17]:
x = model._pretrained_one(block4_pool_features)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    elements = sess.run(x)

In [18]:
elements[0,...]

array([0.5991738 , 0.39890918, 0.54884744, 0.58491796, 0.49875844,
       0.6633831 , 0.4610215 , 0.48753986, 0.55230516, 0.73752356,
       0.6895586 , 0.5144405 , 0.68510246, 0.55465204, 0.67978007,
       0.5118008 , 0.3712615 , 0.4681861 , 0.43286905, 0.5790298 ,
       0.511032  , 0.45862073, 0.6345001 , 0.38931087, 0.5389054 ,
       0.57338625, 0.5118364 , 0.45663428, 0.6264829 , 0.39470193,
       0.4903332 , 0.5296766 , 0.42885098, 0.45153227, 0.4189087 ,
       0.5063828 , 0.40658584, 0.40025932, 0.4886654 , 0.44674605,
       0.51520175, 0.43246642, 0.39662555, 0.55826545, 0.343367  ,
       0.5327681 , 0.49816585, 0.5795686 , 0.5163955 , 0.65860146,
       0.38446668, 0.49901688, 0.5851726 , 0.4817238 , 0.40790725,
       0.58252823, 0.6432604 , 0.53558576, 0.6118953 , 0.4323362 ,
       0.4872922 , 0.48305306, 0.5456262 , 0.5101642 , 0.6063409 ,
       0.6129402 , 0.4272153 , 0.5265189 , 0.6553963 , 0.5552257 ,
       0.51442426, 0.41636285, 0.7153402 , 0.4962433 , 0.58740

In [22]:
x = model._pretrained_two(block4_pool_features)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    elements = sess.run(x)

In [23]:
elements[0,...]

array([0.00400459, 0.01085337, 0.02087608, 0.0149787 , 0.01605831,
       0.00466571, 0.01013105, 0.01187607, 0.01378341, 0.01127759,
       0.00506886, 0.00875251, 0.00623035, 0.01638601, 0.01004365,
       0.00795197, 0.01034357, 0.02756001, 0.01510367, 0.00900358,
       0.00510436, 0.00578937, 0.0050557 , 0.01797385, 0.00707288,
       0.00994353, 0.01236697, 0.01513159, 0.00756992, 0.00994322,
       0.01121983, 0.00547572, 0.01047068, 0.01461672, 0.01162347,
       0.01621258, 0.00671258, 0.01428959, 0.00748722, 0.00386721,
       0.00729972, 0.00924714, 0.00977064, 0.01026657, 0.00787268,
       0.00774136, 0.00625807, 0.0154231 , 0.01169518, 0.00450689,
       0.00688122, 0.00347036, 0.02989487, 0.01126648, 0.01189918,
       0.01644664, 0.01346847, 0.00728928, 0.00644581, 0.01332345,
       0.00658745, 0.00913884, 0.01161848, 0.00619993, 0.00451522,
       0.00575432, 0.00605281, 0.0162519 , 0.01543112, 0.00790877,
       0.0061468 , 0.00544485, 0.00830282, 0.01603138, 0.01023